***
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

## Pre-Knowledge Graph Build Ontology Download and Cleaning
***
***

**Authors:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com), [ECavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@unimi.it)    
**GitHub Repositories:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki), [RNA-KG](https://github.com/AnacletoLAB/RNA-KG/)
  

## Purpose

This notebook serves as a script to help prepare ontologies prior to be ingested into the knowledge graph build algorithm. This script performs the following steps:  

1. [Download Ontologies](#download-ontologies)  
2. [Clean Ontologies](#clean-ontologies)  
3. [Merge Ontologies](#merge-ontologies)
4. [Create Merged Ontologies Graph](#graph-ontologies)

**Dependencies:**   
- <u>Scripts</u>: This notebook utilizes several helper functions from the following scripts:  
  - [utility scripts](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils)  
  - [ontology_cleaning.py](https://github.com/callahantiff/PheKnowLator/blob/master/builds/ontology_cleaning.py) 
- <u>Software</u>: [OWLTools](https://github.com/owlcollab/owltools)  
- <u>Data</u>: All downloaded and generated data sources are provided through [10.5281/zenodo.10078876](https://zenodo.org/doi/10.5281/zenodo.10078876) dedicated repository. <u>This notebook will download everything that is needed for you</u>. 

## Set-Up Environment
_____

In [1]:
%%capture
import sys
!{sys.executable} -m pip install -r requirements.txt
sys.path.append('../')

In [ ]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import os
import pickle
import re
import requests
import tarfile
import shutil
import pandas
import pandas as pd
import gffpandas.gffpandas as gffpd
import numpy as np
pd.set_option('display.max_columns', None)
import re
import zipfile
import io

from collections import Counter
from functools import reduce
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import OWL, RDF, RDFS
from reactome2py import content
from tqdm import tqdm
from typing import Dict

from pkt_kg.utils import * 
from builds.ontology_cleaning import *

from Bio import SeqIO, Entrez

from Bio.SeqIO.FastaIO import SimpleFastaParser

from typing import Tuple

#### Define Global Variables

In [4]:
# directory to store resources
resource_data_location = '../resources/'    

# directory to use for unprocessed data
unprocessed_data_location = '../resources/processed_data/unprocessed_data/'

# directory to use for processed data
processed_data_location = '../resources/processed_data/'

# directory to write relations data to
relations_data_location = '../resources/relations_data/'

# directory to write ontology data to
ontology_data_location = '../resources/ontologies/'

# directory to write edges data to
edge_data_location = '../resources/edge_data/'

# processed data url 
processed_url = 'https://storage.googleapis.com/pheknowlator/current_build/data/processed_data/'

# owltools location
owltools_location = '../pkt_kg/libs/owltools'

<br>

***
### DOWNLOAD ONTOLOGIES <a class="anchor" id="download-ontologies"></a>
***

In [ ]:
onto_list = ['chebi', 'pr', 'ro', 'mondo', 'go/extensions/go-plus', 'pw', 'so', 'hp/hp-international', 'uberon', 'vo', 'clo']
# If pr and chebi servers are down (occurs frequently), download the ontologies from the following URLs:
# pr --> http://web.archive.org/web/20240926155115/https://proconsortium.org/download/current/pro_reasoned.owl
# chebi --> ftp://ftp.ebi.ac.uk/pub/databases/chebi/ontology/chebi.owl
command = '{} {} --merge-import-closure -o {}'
for ontology in onto_list:
    print('Processing ' + ontology + ' ontology')
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/' + ontology + '.owl',
                             ontology_data_location + ontology + '_with_imports.owl'))

# For compatibility with the PheKnowLator ecosystem, we rename the ontology files to match the naming convention
os.rename(os.path.join(ontology_data_location,
                       'go/extensions/go-plus_with_imports.owl'), os.path.join(ontology_data_location, 'go_with_imports.owl'))
os.rename(os.path.join(ontology_data_location,
                       'hp/hp-international_with_imports.owl'), os.path.join(ontology_data_location, 'hp_with_imports.owl'))
os.rename(os.path.join(ontology_data_location, 'uberon_with_imports.owl'), os.path.join(ontology_data_location, 'ext_with_imports.owl'))

# Remove empty directories
os.rmdir(os.path.join(ontology_data_location, 'hp'))
os.rmdir(os.path.join(ontology_data_location, 'go/extensions'))
os.rmdir(os.path.join(ontology_data_location, 'go'))

We create a version of the PRotein Ontology that contains only human and viral proteins.

In [ ]:
networkx_mdg: networkx.MultiDiGraph = networkx.MultiDiGraph()
    
pr_graph = Graph().parse(ontology_data_location + 'pr_with_imports.owl')
print('There are {} axioms in the ontology (date: {})'.format(len(pr_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

for s, p, o in tqdm(pr_graph):
    networkx_mdg.add_edge(s, o, **{'key': p})

In [ ]:
obo = Namespace('http://purl.obolibrary.org/obo/')

human_classes_restriction = list(pr_graph.triples((None, OWL.someValuesFrom, obo.NCBITaxon_9606)))
human_classes = [list(pr_graph.subjects(RDFS.subClassOf, x[0])) for x in human_classes_restriction]
human_pro_classes = list(str(i) for j in human_classes for i in j if 'PR_' in str(i))

print('There are {} edges in the ontology (date:{})'.format(len(human_pro_classes), datetime.datetime.now().strftime('%m/%d/%Y')))
human_classes[:5]

In [ ]:
# We extract the list of virus taxa to extract terms connected to them (as with Homo Sapiens -- NCBITaxon_9606)
viruses_NCBI = list(pr_graph.subjects(RDFS.subClassOf, obo.NCBITaxon_10239))
viral_classes_restriction = []
for i in viruses_NCBI :
    viral_classes_restriction.append(list(pr_graph.triples((None, OWL.someValuesFrom, i))))

temp = []
for i in viral_classes_restriction:
    for j in i:
        temp.append(j)
viral_classes_restriction = temp
viral_classes = [list(pr_graph.subjects(RDFS.subClassOf, x[0])) for x in viral_classes_restriction]
# remove empty classes
viral_classes = [x for x in viral_classes if x]
viral_pro_classes = list(str(i) for j in viral_classes for i in j if 'PR_' in str(i))
print('There are {} edges in the ontology (date:{})'.format(len(viral_pro_classes), datetime.datetime.now().strftime('%m/%d/%Y')))
viral_classes[:5]

In [ ]:
human_pro_classes = human_pro_classes + viral_pro_classes

# create a new graph using bfs paths
human_pro_graph = Graph()
human_networkx_mdg = networkx.MultiDiGraph()

for node in tqdm(human_pro_classes):
    forward = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='original'))
    reverse = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='reverse'))
    
    # add edges from forward and reverse bfs paths
    for path in set(forward + reverse):
        human_pro_graph.add((path[0], path[2], path[1]))
        human_networkx_mdg.add_edge(path[0], path[1], **{'key': path[2]})

In [ ]:
# get connected component information
print('Finding Connected Components')
components = list(networkx.connected_components(human_networkx_mdg.to_undirected()))
component_dict = sorted(components, key=len, reverse=True)

# if more than 1 connected component, only keep the biggest
if len(component_dict) > 1:
    print('Cleaning Graph: Removing Small Disconnected Components')
    for node in tqdm([x for y in component_dict[1:] for x in list(y)]):
        human_pro_graph.remove((node, None, None))

# save data
print('Saving Human Subset of the Protein Ontology')
human_pro_graph.serialize(destination=unprocessed_data_location + 'human_pro.owl', format='xml')

In [ ]:
# run reasoner to ensure that there are no incomplete triples or inconsistent classes
command = '{} {} --reasoner {} --run-reasoner --assert-implied -o {}'
os.system(command.format(owltools_location, unprocessed_data_location + 'human_pro.owl', 'elk',
                         ontology_data_location + 'pr_with_imports.owl'))

**Genomic Typing Dictionary**  
Read in the  `genomic_typing_dict.pkl` dictionary, which is needed in order to preprocess the genomic identifier datasets.

In [ ]:
# download data
url = 'https://zenodo.org/records/10056198/files/genomic_typing_dict.pkl.zip?download=1'
data_downloader(url, unprocessed_data_location)

In [18]:
# load data
genomic_type_mapper = pickle.load(open(unprocessed_data_location + 'genomic_typing_dict.pkl', 'rb'))

***
**HGNC Data** 

_Human Gene Set Data_ - `hgnc_complete_set.txt`

In [ ]:
# download data
url = 'https://storage.googleapis.com/public-download-files/hgnc/tsv/tsv/hgnc_complete_set.txt'
data_downloader(url, unprocessed_data_location)

In [ ]:
# load data
hgnc = pandas.read_csv(unprocessed_data_location + 'hgnc_complete_set.txt', header=0, delimiter='\t', low_memory=False)
hgnc.head()

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be protein-coding, other or ncRNA.

In [ ]:
hgnc = hgnc.loc[hgnc['status'].apply(lambda x: x == 'Approved')]
hgnc = hgnc[['hgnc_id', 'entrez_id', 'ensembl_gene_id', 'uniprot_ids', 'symbol', 'locus_type', 'alias_symbol', 'name', 'location', 'alias_name']]
hgnc.rename(columns={'uniprot_ids': 'uniprot_id', 'location': 'map_location', 'locus_type': 'hgnc_gene_type'}, inplace=True)
hgnc['hgnc_id'].replace('.*\:', '', inplace=True, regex=True)  # strip 'HGNC' off of the identifiers
hgnc.fillna('None', inplace=True)  # replace NaN with 'None'
hgnc['entrez_id'] = hgnc['entrez_id'].apply(lambda x: str(int(x)) if x != 'None' else 'None')  # make col str

# combine certain columns into single column
hgnc['name'] = hgnc['name'] + '|' + hgnc['alias_name']
hgnc['synonyms'] = hgnc['alias_symbol'] + '|' + hgnc['alias_name'] + '|' + hgnc['name']
hgnc['symbol'] = hgnc['symbol'] + '|' + hgnc['alias_symbol']

# explode nested data and reformat values in preparation for combining it with other gene identifiers
explode_df_hgnc = explodes_data(hgnc.copy(), ['ensembl_gene_id', 'uniprot_id', 'symbol', 'name', 'synonyms'], '|')

# reformat hgnc gene type
for val in genomic_type_mapper['hgnc_gene_type'].keys():
    explode_df_hgnc['hgnc_gene_type'].replace(val, genomic_type_mapper['hgnc_gene_type'][val], inplace=True)

# reformat master hgnc gene type
explode_df_hgnc['master_gene_type'] = explode_df_hgnc['hgnc_gene_type']
master_dict = genomic_type_mapper['hgnc_master_gene_type']
for val in master_dict.keys():
    explode_df_hgnc['master_gene_type'].replace(val, master_dict[val], inplace=True)

# post-process reformatted data
explode_df_hgnc.drop(['alias_symbol', 'alias_name'], axis=1, inplace=True)  # remove original gene type column
explode_df_hgnc.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_hgnc.head(n=3)

***
**Ensembl Data**

_Human Gene Set Data_ - `Homo_sapiens.GRCh38.113.gtf.gz`

In [ ]:
# download data
url = 'ftp://ftp.ensembl.org/pub/release-113/gtf/homo_sapiens/Homo_sapiens.GRCh38.113.gtf.gz'
data_downloader(url, unprocessed_data_location)

In [ ]:
# load data
ensembl_geneset = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.113.gtf',
                                  header = None, delimiter='\t', skiprows=5, usecols=[8], low_memory=False)
ensembl_geneset.head()

_Preprocess Data_  
Data file needs to be reformatted in order for it to be able to be merged with the other gene, RNA, and protein identifier data. To do this, we iterate over each row of the data and extract the fields shown below in `column_names`, making each of these extracted fields their own column. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
ensembl_data = list(ensembl_geneset[8]); ensembl_df_data = []
for i in tqdm(range(0, len(ensembl_data))):
    if 'gene_id' in ensembl_data[i] and 'transcript_id' in ensembl_data[i]:
        row_dict = {x.split(' "')[0].lstrip(): x.split(' "')[1].strip('"') for x in ensembl_data[i].split(';')[0:-1]}
        if 'gene_name' not in row_dict:
            row_dict['gene_name'] = 'None'
        if 'gene_biotype' not in row_dict:
            row_dict['gene_biotype'] = 'None'
        if 'transcript_name' not in row_dict:
            row_dict['transcript_name'] = 'None'
        if 'transcript_biotype' not in row_dict:
            row_dict['transcript_biotype'] = 'None'
        ensembl_df_data += [(row_dict['gene_id'], row_dict['transcript_id'], row_dict['gene_name'],
                           row_dict['gene_biotype'], row_dict['transcript_name'], row_dict['transcript_biotype'])]
# convert to data frame
ensembl_geneset = pandas.DataFrame(ensembl_df_data,
                                   columns=['ensembl_gene_id', 'transcript_stable_id', 'symbol',
                                            'ensembl_gene_type', 'transcript_name', 'ensembl_transcript_type'])

# reformat ensembl gene type
gene_dict = genomic_type_mapper['ensembl_gene_type']
for val in gene_dict.keys(): ensembl_geneset['ensembl_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master gene type
ensembl_geneset['master_gene_type'] = ensembl_geneset['ensembl_gene_type']
gene_dict = genomic_type_mapper['ensembl_master_gene_type']
for val in gene_dict.keys(): ensembl_geneset['master_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master transcript type
ensembl_geneset['ensembl_transcript_type'].replace('vault_RNA', 'vaultRNA', inplace=True, regex=False)
ensembl_geneset['master_transcript_type'] = ensembl_geneset['ensembl_transcript_type']
trans_dict = genomic_type_mapper['ensembl_master_transcript_type']
for val in trans_dict.keys(): ensembl_geneset['master_transcript_type'].replace(val, trans_dict[val], inplace=True)

# post-process reformatted data
ensembl_geneset.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_geneset.head(n=3)

***
**Ensembl Annotation Data**

_Ensembl-UniProt_ - `Homo_sapiens.GRCh38.113.uniprot.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-uniprot` mapping file. These files are vital for successfully merging the ensembl identifiers with the uniprot data set.

In [ ]:
# download data
url_uniprot = 'ftp://ftp.ensembl.org/pub/release-113/tsv/homo_sapiens/Homo_sapiens.GRCh38.113.uniprot.tsv.gz'
data_downloader(url_uniprot, unprocessed_data_location)

# load data
ensembl_uniprot = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.113.uniprot.tsv', header=0, delimiter='\t', low_memory=False)

# preprocess data
ensembl_uniprot.rename(columns={'xref': 'uniprot_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)
ensembl_uniprot.replace('-', 'None', inplace=True)
ensembl_uniprot.fillna('None', inplace=True)
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['xref_identity'].apply(lambda x: x != 'None')]
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['uniprot_id'].apply(lambda x: '-' not in x)]  # remove isoforms
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['info_type'].apply(lambda x: x == 'DIRECT')]
# ensembl_uniprot['master_gene_type'] = ['protein-coding'] * len(ensembl_uniprot)
# ensembl_uniprot['master_transcript_type'] = ['protein-coding'] * len(ensembl_uniprot)
ensembl_uniprot.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)
ensembl_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)
ensembl_uniprot.head()

_Ensembl-Entrez_ - `Homo_sapiens.GRCh38.113.entrez.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-entrez` mapping file. These files are vital for successfully merging the ensembl identifiers with the entrez data set.

In [ ]:
# download data
url_entrez = 'ftp://ftp.ensembl.org/pub/release-113/tsv/homo_sapiens/Homo_sapiens.GRCh38.113.entrez.tsv.gz'
data_downloader(url_entrez, unprocessed_data_location)

# load data
ensembl_entrez = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.113.entrez.tsv', header=0, delimiter='\t', low_memory=False)

# preprocess data
ensembl_entrez.rename(columns={'xref': 'entrez_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)
ensembl_entrez = ensembl_entrez.loc[ensembl_entrez['db_name'].apply(lambda x: x == 'EntrezGene')]
ensembl_entrez = ensembl_entrez.loc[ensembl_entrez['info_type'].apply(lambda x: x == 'DEPENDENT')]
ensembl_entrez.replace('-', 'None', inplace=True)
ensembl_entrez.fillna('None', inplace=True)
ensembl_entrez.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)
ensembl_entrez.drop_duplicates(subset=None, keep='first', inplace=True)
ensembl_entrez.head()

_Merge Annotation Data_ - `ensembl_uniprot` + `ensembl_entrez`

In [ ]:
merge_cols = list(set(ensembl_entrez).intersection(set(ensembl_uniprot)))
ensembl_annot = pandas.merge(ensembl_uniprot, ensembl_entrez, on=merge_cols, how='outer')
ensembl_annot.fillna('None', inplace=True)

# preview data
ensembl_annot.head(n=3)

_Merge Ensembl Annotation and Gene Set Data_ - `ensembl_geneset` + `ensembl_annot`

In [ ]:
merge_cols = list(set(ensembl_annot).intersection(set(ensembl_geneset)))
ensembl = pandas.merge(ensembl_geneset, ensembl_annot, on=merge_cols, how='outer')
ensembl.fillna('None', inplace=True)
ensembl.replace('NA','None', inplace=True, regex=False)

# preview data
ensembl.head(n=3)

_Save Cleaned Ensembl Data_  
Save the cleaned Ensembl data so that it can be used when generating node metadata for transcript identifiers.

In [30]:
ensembl.to_csv(processed_data_location + 'ensembl_identifier_data_cleaned.txt', header=True, sep='\t', index=False)

***
**UniProt Data**   
_Human Gene Set Data_ - `uniprot_identifier_mapping.tab`

This data was obtained by querying the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) using the *organism:"Homo sapiens (Human) [9606]"* keyword and including the following columns:
- Entry (Standard)    
- GeneID (*Genome Annotation*)  
- Ensembl (*Genome Annotation*)  
- HGNC (*Organism-specific*)  
- Gene names (primary) (*Names & Taxonomy*)    
- Gene synonym (primary) (*Names & Taxonomy*)    

The URL to access the results of this query is obtained by clicking on the share symbol and copying the free-text from the box. To obtain the data in a tab-delimited format the following string is appended to the end of the URL: "&format=tab".

**NOTE.** Be sure to obtain a new URL from the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) when rebuilding to ensure you are getting the most up-to-date data. This query was last generated on `01/02/2025`.

In [ ]:
# download data
url = 'https://rest.uniprot.org/uniprotkb/stream?fields=accession%2Cid%2Cxref_geneid%2Cxref_ensembl%2Cxref_hgnc%2Cgene_primary%2Cgene_synonym%2Corganism_id%2Creviewed&format=tsv&query=%28%28organism_id%3A9606%29%29'
data_downloader(url, unprocessed_data_location, 'uniprot_identifier_mapping.tab')

# load data
uniprot = pandas.read_csv(unprocessed_data_location + 'uniprot_identifier_mapping.tab', header=0, delimiter='\t')
uniprot.head()

In [32]:
uniprot.drop(columns=['Organism (ID)'], inplace=True)

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, and unnesting `"|"` delimited data.

In [ ]:
uniprot.fillna('None', inplace=True)  # replace NaN with 'None'
uniprot.rename(columns={'Entry': 'uniprot_id',
                        'GeneID': 'entrez_id',
                        'Ensembl': 'transcript_stable_id',
                        'HGNC': 'hgnc_id',
                        'Gene Names (synonym)': 'synonyms',
                        'Gene Names (primary)' :'symbol',
                        'Reviewed': 'Status'}, inplace=True)

# update space-delimited synonyms to a pipe (i.e. '|')
uniprot['synonyms'] = uniprot['synonyms'].apply(lambda x: '|'.join(x.split()) if x.isupper() else x)

# only keep reviewed entries
uniprot = uniprot.loc[uniprot['Status'].apply(lambda x: x != 'unreviewed')]

# explode nested data
explode_df_uniprot = explodes_data(uniprot.copy(), ['transcript_stable_id', 'entrez_id', 'hgnc_id'], ';')
explode_df_uniprot = explodes_data(explode_df_uniprot.copy(), ['symbol', 'synonyms'], '|')

# strip out uniprot names
explode_df_uniprot['transcript_stable_id'].replace('\s.*','', inplace=True, regex=True)

# remove duplicates
explode_df_uniprot.drop(['Status'], axis=1, inplace=True)
explode_df_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_uniprot.head(n=3)

***
**NCBI Data**   
_Human Gene Set Data_ - `Homo_sapiens.gene_info`

In [ ]:
# download data
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz'
data_downloader(url, unprocessed_data_location)

# load data
ncbi_gene = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.gene_info', header=0, delimiter='\t', low_memory=False)

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. Then, the `gene_type` variable is cleaned such that each of the variable's values are re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
# preprocess data
ncbi_gene = ncbi_gene.loc[ncbi_gene['#tax_id'].apply(lambda x: x == 9606)]  # remove non-human rows
ncbi_gene.replace('-', 'None', inplace=True)
ncbi_gene.rename(columns={'GeneID': 'entrez_id', 'Symbol': 'symbol', 'Synonyms': 'synonyms'}, inplace=True)
ncbi_gene['synonyms'] = ncbi_gene['synonyms'] + '|' + ncbi_gene['description'] + '|' + ncbi_gene['Full_name_from_nomenclature_authority'] + '|' + ncbi_gene['Other_designations']
ncbi_gene['symbol'] = ncbi_gene['Symbol_from_nomenclature_authority'] + '|' + ncbi_gene['symbol']
ncbi_gene['name'] = ncbi_gene['Full_name_from_nomenclature_authority'] + '|' + ncbi_gene['description']

# explode nested data
explode_df_ncbi_gene = explodes_data(ncbi_gene.copy(), ['symbol', 'synonyms', 'name', 'dbXrefs'], '|')

# clean up results
explode_df_ncbi_gene['entrez_id'] = explode_df_ncbi_gene['entrez_id'].astype(str)
explode_df_ncbi_gene = explode_df_ncbi_gene.loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.split(':')[0] in ['Ensembl', 'HGNC', 'IMGT/GENE-DB'])]
explode_df_ncbi_gene['hgnc_id'] = explode_df_ncbi_gene['dbXrefs'].loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.startswith('HGNC'))]
explode_df_ncbi_gene['ensembl_gene_id'] = explode_df_ncbi_gene['dbXrefs'].loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.startswith('Ensembl'))]
explode_df_ncbi_gene.fillna('None', inplace=True)

# reformat entrez gene type
explode_df_ncbi_gene['entrez_gene_type'] = explode_df_ncbi_gene['type_of_gene']
gene_dict = genomic_type_mapper['entrez_gene_type']
for val in gene_dict.keys(): explode_df_ncbi_gene['entrez_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master gene type
explode_df_ncbi_gene['master_gene_type'] = explode_df_ncbi_gene['entrez_gene_type']
gene_dict = genomic_type_mapper['master_gene_type']
for val in gene_dict.keys(): explode_df_ncbi_gene['master_gene_type'].replace(val, gene_dict[val], inplace=True)

# post-process reformatted data
explode_df_ncbi_gene.drop(['type_of_gene', 'dbXrefs', 'description', 'Nomenclature_status', 'Modification_date',
                           'LocusTag', '#tax_id', 'Full_name_from_nomenclature_authority', 'Feature_type',
                           'Symbol_from_nomenclature_authority'], axis=1, inplace=True)
explode_df_ncbi_gene['hgnc_id'] = explode_df_ncbi_gene['hgnc_id'].replace('HGNC:', '', regex=True)
explode_df_ncbi_gene['ensembl_gene_id'] = explode_df_ncbi_gene['ensembl_gene_id'].replace('Ensembl:', '', regex=True)
explode_df_ncbi_gene.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_ncbi_gene.head(n=3)

***
**Protein Ontology Identifier Mapping Data**   
_Protein Ontology Identifier Data_ - `promapping.txt`

In [ ]:
# download data
url = 'https://proconsortium.org/download/current/promapping.txt'
data_downloader(url, unprocessed_data_location)

# load data
pro_map = pandas.read_csv(unprocessed_data_location + 'promapping.txt', header=None, names=['pro_id', 'entry', 'pro_mapping'], delimiter='\t')

_Preprocess Data_  
Basic filtering to to include `Protein Ontology` mappings to `Uniprot` identifiers and cleaning to update formatting of accession values (i.e. removing `UniProtKB:`).

In [ ]:
pro_map = pro_map.loc[pro_map['entry'].apply(lambda x: x.startswith('Uni') and '_VAR' not in x and ', ' not in x)]  # keep 'UniProtKB' rows
pro_map = pro_map.loc[pro_map['pro_mapping'].apply(lambda x: x.startswith('exact'))] # keep exact mappings
pro_map['pro_id'].replace('PR:','PR_', inplace=True, regex=True)  # replace PR: with PR_
pro_map['entry'].replace('(^\w*\:)','', inplace=True, regex=True)  # remove id prefixes
pro_map = pro_map.loc[pro_map['pro_id'].apply(lambda x: '-' not in x)] # remove isoforms
pro_map.rename(columns={'entry': 'uniprot_id'}, inplace=True)  # rename columns before merging
pro_map.drop(['pro_mapping'], axis=1, inplace=True)  # remove uneeded columns
pro_map.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
pro_map.head(n=3)

***

#### Merging Processed Genomic Identifier Data Sources  
Merging all of the genomic identifier data sources is needed in order to create a map that can be used to integrate the different genomic data sources.

***
*Data Sources:* `hgnc` + `ensembl`

In [ ]:
merge_cols = list(set(explode_df_hgnc.columns).intersection(set(ensembl.columns)))
ensembl_hgnc_merged_data = pandas.merge(ensembl, explode_df_hgnc, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_merged_data.fillna('None', inplace=True)
ensembl_hgnc_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_merged_data` + `explode_df_uniprot`

In [ ]:
merge_cols = list(set(ensembl_hgnc_merged_data.columns).intersection(set(explode_df_uniprot.columns)))
ensembl_hgnc_uniprot_merged_data = pandas.merge(ensembl_hgnc_merged_data, explode_df_uniprot, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_uniprot_merged_data.fillna('None', inplace=True)
ensembl_hgnc_uniprot_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_uniprot_merged_data` + `Homo_sapiens.gene_info`

In [ ]:
merge_cols = merge_cols = list(set(ensembl_hgnc_uniprot_merged_data).intersection(set(explode_df_ncbi_gene.columns)))
ensembl_hgnc_uniprot_ncbi_merged_data = pandas.merge(ensembl_hgnc_uniprot_merged_data, explode_df_ncbi_gene, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_uniprot_ncbi_merged_data.fillna('None', inplace=True)
ensembl_hgnc_uniprot_ncbi_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_ncbi_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_uniprot_ncbi_merged_data` + `promapping.txt`  

In [ ]:
merged_data = pandas.merge(ensembl_hgnc_uniprot_ncbi_merged_data, pro_map, on='uniprot_id', how='outer')

# clean up merged data
merged_data.fillna('None', inplace=True)
merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
merged_data.head(n=3)

_Fix Symbol Formatting_  
some genes are formatted similarly to dates (e.g. `DEC1`), which can be erroneously re-formatted during input as a date value (i.e. `1-DEC`). In order for the data to be successfully merged with other data sources, all date-formatted genes need to be resolved.

In [ ]:
clean_dates = []
for x in tqdm(list(merged_data['symbol'])):
    if '-' in x and len(x.split('-')[0]) < 3 and len(x.split('-')[1]) == 3:
        clean_dates.append(x.split('-')[1].upper() + x.split('-')[0])
    else: clean_dates.append(x)

# add cleaned date var back to data set
merged_data['symbol'] = clean_dates
merged_data.fillna('None', inplace=True)

# make sure that all gene and transcript type colunmns have none recoded to unknown or not protein-coding
merged_data['hgnc_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['ensembl_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['entrez_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['master_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['master_transcript_type'].replace('None', 'not protein-coding', inplace=True, regex=False)
merged_data['ensembl_transcript_type'].replace('None', 'unknown', inplace=True, regex=False)

# remove duplicates
merged_data_clean = merged_data.drop_duplicates(subset=None, keep='first')

# write data
merged_data_clean.to_csv(processed_data_location + 'Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt', header=True, sep='\t', index=False)
    
# preview data
merged_data_clean.head(n=3)

***
**Create a Master Mapping Dictionary**  
Although the above steps result in a `pandas.Dataframe` of the merged identifiers, there is still work needed in order to be able to obtain a complete mapping between the identifiers. For example, if you were to search for Entrez gene identifier `entrez_259234` you would find the following mappings: `entrez_259234-ENSG00000233316`, `entrez_259234-DSCR10`. If you only had `ENSG00000233316`, with the current data you would be unable to obtain the gene symbol without first mapping to the Entrez gene identifier. 

To solve this problem, we build a master dictionary where the keys are `ensembl_gene_id`, `transcript_stable_id`, `protein_stable_id`, `uniprot_id`, `entrez_id`, `hgnc_id`, `pro_id`, and `symbol` identifiers and values are the list of genomic identifiers that match to each identifier. It's important to note that there are several labeling identifiers (i.e. `name`, `chromosome`, `map_location`, `Other_designations`, `synonyms`, `transcript_name`, `*_gene_types`, and `trasnscript_type_update`), which will only be mapped when clustered against one of the primary identifier types (i.e. the keys described above).

_Note_. The next chunk does a lot of heavy lifting and takes approximately ~40 minutes to run.

In [43]:
# reformat data to convert all nones, empty values, and unknowns to NaN
for col in merged_data_clean.columns:
    merged_data_clean[col] = merged_data_clean[col].apply(lambda x: '|'.join([i for i in x.split('|') if i != 'None']))
merged_data_clean.replace(to_replace=['None', '', 'unknown'], value=numpy.nan, inplace=True)
identifiers = [x for x in merged_data_clean.columns if x.endswith('_id')] + ['symbol']

In [ ]:
# convert data to dictionary
master_dict = {}
for idx in tqdm(identifiers):
    grouped_data = merged_data_clean.groupby(idx)
    grp_ids = set([x for x in list(grouped_data.groups.keys()) if x != numpy.nan])
    for grp in grp_ids:
        df = grouped_data.get_group(grp).dropna(axis=1, how='all')
        df_cols, key = df.columns, idx + '_' + grp
        val_df = [[col + '_' + x for x in set(df[col]) if isinstance(x, str)] for col in df_cols if col != idx]
        if len(val_df) > 0:
            if key in master_dict.keys(): master_dict[key] += [i for j in val_df for i in j if len(i) > 0]
            else: master_dict[key] = [i for j in val_df for i in j if len(i) > 0]  

_Finalizing Master Mapping Dictionary_  
Then, we need to identify a master gene and transcript type for each entity because the last ran code chunk can result in several genes and transcripts with differing types (i.e. `protein-coding` or `not protein-coding`). The next step collects all information for each gene and transcript and performs a voting procedure to select a single primary gene and transcript type.

In [ ]:
reformatted_mapped_identifiers = dict()
for key, values in tqdm(master_dict.items()):
    identifier_info = set(values); gene_prefix = 'master_gene_type_'; trans_prefix = 'master_transcript_type_'
    if key.split('_')[0] in ['protein', 'uniprot', 'pro']: pass
    elif 'transcript' in key:
        trans_match = [x.replace(trans_prefix, '') for x in values if trans_prefix in x]
        if len(trans_match) > 0:
            t_type_list = ['protein-coding' if ('protein-coding' in trans_match or 'protein_coding' in trans_match) else 'not protein-coding']
            identifier_info |= {'transcript_type_update_' + max(set(t_type_list), key=t_type_list.count)}
    else:
        gene_match = [x.replace(gene_prefix, '') for x in values if x.startswith(gene_prefix) and 'type' in x]
        if len(gene_match) > 0:
            g_type_list = ['protein-coding' if ('protein-coding' in gene_match or 'protein_coding' in gene_match) else 'not protein-coding']
            identifier_info |= {'gene_type_update_' + max(set(g_type_list), key=g_type_list.count)}
    reformatted_mapped_identifiers[key] = identifier_info

In [46]:
# save a copy of the dictionary
# output > 4GB requires special approach: https://stackoverflow.com/questions/42653386/does-pickle-randomly-fail-with-oserror-on-large-files
filepath = processed_data_location + 'Merged_gene_rna_protein_identifiers.pkl'

# defensive way to write pickle.write, allowing for very large files on all platforms
max_bytes, bytes_out = 2**31 - 1, pickle.dumps(reformatted_mapped_identifiers)
n_bytes = sys.getsizeof(bytes_out)

with open(filepath, 'wb') as f_out:
    for idx in range(0, n_bytes, max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

<br>

***
### CLEAN ONTOLOGIES <a class="anchor" id="clean-ontologies"></a>
***

The ontology cleaning step includes the following error checks, each of which are explained below and each of which are applied to individual ontologies, the set of merged ontologies or both: (1) Value Errors, (2) Identifier Errors, (3) Duplicate and Obsolete Entities, (4) Punning Errors, and (5) Entity Normalization Errors.

<br>

### Value Errors  
*** 
**Level:** `individual ontology`; `merged-ontology`    

**Description:** This check utilizes the [`owlready2`](https://pypi.org/project/Owlready2/) library to read in each of the ontologies. This library is strict and will catch a wide variety of value errors. 

**Solution:** Parse the error message using the provided `ErrorType` and line number and repair it. For `ValueErrors` incorrectly typed input are re-typed.

*Example Findings*  
The [Cell Line Ontology](http://www.clo-ontology.org/) yield the following error message:

```python
ValueError: invalid literal for int() with base 10: '永生的乳腺衍生细胞系细胞'
...
OwlReadyOntologyParsingError: RDF/XML parsing error in file clo_with_imports.owl, line 10970, column 99.
```

This tells us that we need to repair the triple containing the Literal '永生的乳腺衍生细胞系细胞' by removing it and redefining it as a `string`, rather than an `int` as it is currently defined as. This is currently noted as an issue in the [Cell Line Ontology's](http://www.clo-ontology.org/) GitHub repo ([issue #48](https://github.com/CLO-ontology/CLO/issues/48)). 

<br>

### Identifier Errors  
*** 
**Level:** `individual ontology`; `merged-ontology`  

**Description:** This check verifies consistency of identifier prefixes. For example, we want to find identifiers that are incorrectly formatted like occurrences of `PRO_XXXXXXX` which should be `PR_XXXXXXX`.

**Solution:** Incorrectly formatted class identifiers are updated. This is a tricky task to do in an automated manner and is something that should be updated if any new ontologies are added to the `PheKnowLator` build. Currently, the code below checks and logs any hits, but only fixes the following known errors: Vaccine Ontology: `PRO` which should be `PR`.

*Example Findings*  
Running this check revealed mislabeling of `2` [pROtein Ontology](https://proconsortium.org/) identifiers in the [Vaccine Ontology](http://www.violinet.org/vaccineontology/) (see [this](https://github.com/vaccineontology/VO/issues/4) GitHub issue).

<br>

### Obsolete and/or Deprecated Entities
*** 
**Level:** `individual ontology`; `merged-ontology`  

**Description:** Verify that the ontology only contains current content.

**Solution:** All obsolete classes and any triples that they participate in are removed from an ontologies.

<br>

### Normalization Errors  
*** 
**Level:** `merged-ontology`

These checks are performed at the individual- and merged-ontology levels. There are two types of checks that are performed:  

<u>Normalize Existing Ontology Classes</u>  
  - **Description:** Checks for inconsistencies in ontology classes that overlap with non-ontology entity identifiers (e.g. if HP includes `HGNC` identifiers, but PheKnowLator utilizes `Entrez` identifiers). 

  - **Solution:** While there are other types of identifiers, we currently focus primarily on resolving errors involving the genomic identifiers, since we have a master dictionary we can use (`Merged_gene_rna_protein_identifiers.pkl` -- which is generated during the data preporcessing steps of the build). This check can be updated in future iterations to include other types of identifiers, but given our detailed examination of the `v2.0.0` ontologies, these were the identifier types that needed repair.

<u>Normalize Duplicate Ontology Concepts</u>  
  - **Description:** Make sure that all classes that represent the same entity are connected to each other. For example, consider the following: the [Sequence Ontology](http://www.sequenceontology.org/), [ChEBI](https://www.ebi.ac.uk/chebi), and [PRotein Ontology](https://proconsortium.org/) all include terms for protein, but none of these classes are connected to each other.

  - **Solution:** Choose a primary concept for all duplicate scenarios and make duplicate concepts an `RDFS:subClassOf` the primary concept. In the future, this check could be improved by leveraging [KBOOM](https://www.biorxiv.org/content/10.1101/048843v3).

*Example Findings*  
The follow classes occur in all of the ontologies used in the current build and have to be normalized so that there are not multiple versions of the same concept:  

- Gene: [VO](http://purl.obolibrary.org/obo/OGG_0000000002)  
  - <u>Solution</u>: Make the `VO` imported `OGG` class a subclass of the `SO` gene term  

- Protein: [SO](http://purl.obolibrary.org/obo/SO_0000104), [PRO](http://purl.obolibrary.org/obo/PR_000000001), [ChEBI](http://purl.obolibrary.org/obo/CHEBI_36080) 
  - <u>Solution</u>: Make the `CHEBI` and `PRO` classes a subclass of the `SO` protein term  
  
- Disorder: [VO](http://purl.obolibrary.org/obo/OGMS_0000045)  
  - <u>Solution</u>: Make the `VO` imported `OGMS` class a subclass of the `MONDO` disease term  

- Antigen: [VO](http://purl.obolibrary.org/obo/OBI_1110034)  
  - <u>Solution</u>: Make the `VO` imported OBI class a subclass of the `CHEBI` antigen term  

- Gelatin: [VO]('http://purl.obolibrary.org/obo/VO_0003030') 
  - <u>Solution</u>: Make the `VO` class a subclass of the `CHEBI` gelatin term 

- Hormone: [VO](http://purl.obolibrary.org/obo/FMA_12278) 
  - <u>Solution</u>: Make the `VO` imported `FMA` class a subclass of the `CHEBI` hormone term

<br>

### Punning Errors 
*** 
**Level:** `individual ontology`; `merged-ontology`

**Description:** [Punning](https://www.w3.org/2007/OWL/wiki/Punning) or redeclaration errors occur for a few different reasons, but the primary or most prevalent cause observed in the ontologies used in `PheKnowLator` is due to an `owl:ObjectProperty` being incorrectly redeclared as an `owl:AnnotationProperty` or an `owl:Class` also being defined as an `OWL:ObjectProperty`. 

**Solution:** Consistent with the solution described [here](https://github.com/oborel/obo-relations/issues/130), for `owl:ObjectProperty` redeclarations we remove all `owl:AnnotationProperty` declarations. For all `owl:Class` redeclarations, we remove all `owl:ObjectProperty` redeclarations. 

*Example Findings* 
The [Cell Line Ontology](http://www.clo-ontology.org/) had 7 object properties that were illegally redeclared and triggered punning errors. More details regarding these errors are shown below. 

```bash
2020-12-03 20:57:15,616 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/RO_0002091 in punning not allowed [Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/RO_0002091>)), Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/RO_0002091>))]
2020-12-03 20:57:15,619 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/BFO_0000062 in punning not allowed [Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/BFO_0000062>)), Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/BFO_0000062>))]
2020-12-03 20:57:15,620 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/BFO_0000063 in punning not allowed [Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/BFO_0000063>)), Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/BFO_0000063>))]
2020-12-03 20:57:15,620 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/RO_0002222 in punning not allowed [Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/RO_0002222>)), Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/RO_0002222>))]
2020-12-03 20:57:15,620 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/RO_0000087 in punning not allowed [Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/RO_0000087>)), Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/RO_0000087>))]
2020-12-03 20:57:15,620 ERROR (OWLOntologyManagerImpl:1138) Illegal redeclarations of entities: reuse of entity http://purl.obolibrary.org/obo/RO_0002161 in punning not allowed [Declaration(ObjectProperty(<http://purl.obolibrary.org/obo/RO_0002161>)), Declaration(AnnotationProperty(<http://purl.obolibrary.org/obo/RO_0002161>))]
```

From this message, we can see that we need to remove the following `owl:ObjectProperty` redeclared to `owl:AnnotationProperty`: `RO_0002091`, `BFO_0000062`, `BFO_0000063`, `RO_0002222`, `RO_0000087`, `RO_0002161`. There were also 2 classes (i.e. `CLO_0054407` and `CLO_0054409`) defined as being a `owl:Class` and an `owl:ObjectProperty`. This is currently noted as an issue in the Cell Line Ontology's GitHub repo [issue #43](https://github.com/CLO-ontology/CLO/issues/43)).

<br>

***  
### Set-Up Environment
***  

In [5]:
# # uncomment and run to install any required modules from notebooks/requirements.txt
# import sys
# !{sys.executable} -m pip install -r requirements.txt

In [4]:
# to ensure builds/*.py files and pkt_kg scripts can be reached from notebooks dir
import sys
sys.path.append('../')

#### Load Needed Modules

In [5]:
# import needed libraries
import datetime
import glob
import pickle
import shutil

from rdflib import Graph
from tqdm import tqdm

# import script containing helper functions
from pkt_kg.utils import * 
from builds.ontology_cleaning import *

#### Set Global Variables

In [8]:
# set up environment variables
write_location = '../resources/ontologies'
knowledge_graphs_location = '../resources/knowledge_graphs'
processed_data_location = '../resources/processed_data/'

# set global namespaces
schema = Namespace('http://www.w3.org/2001/XMLSchema#')
obo = Namespace('http://purl.obolibrary.org/obo/')
oboinowl = Namespace('http://www.geneontology.org/formats/oboInOwl#')

#### Helper Functions

In [8]:
# functions needed for processing ontologies
def logically_verifies_cleaned_ontologies(graph, temp_dir, file_location, owltools_location):
    """Logically verifies an ontology by running the ELK deductive logic reasoner. Before running the reasoner
    the instantiated RDFLib object is saved locally.

    Args:
        graph: An RDFLib Graph object containing data.
        temp_dir: A string specifying where where to read from and write to.
        file_location: The name of the file to read and write to in the temp_dir directory.
        owltools_location: A string specifying the location of OWLTOOLs (included in pkt_kg no need to download).
    
    Returns:
        None.
    """

    print('Logically Verifying Ontology')

    # save graph in order to run reasoner
    filename = temp_dir + '/' + file_location
    graph.serialize(destination=filename, format='xml')
    
    # run reasoner
    command = "{} {} --reasoner {} --run-reasoner --assert-implied -o {}"
    return_code = os.system(command.format(owltools_location, filename, 'elk', filename))
    if return_code != 0: raise ValueError('Reasoner Finished with Errors.')

    return None

<br>

***
### INDIVIDUAL ONTOLOGIES <a class="anchor" id="individual-ontologies"></a>
***

**Purpose:** This section focuses on cleaning the individual ontologies which consists of fixing: (1) Parsing Errors; (2) Identifier Errors; (3) Deprecated and Obsolete Classes; and (4) Punning Errors.


**Inputs:** A directory (`write_location`) containing ontology files (`.owl`)

**Outputs:** A directory (`write_location`) containing cleaned ontology files (`.owl`)  


***

### ⚡ Important ⚡

The `OWL API`, when running the [ELK reasoner](), seems to add back some of the errors that this script removes.

- <u>Example 1</u>: In the Vaccine Ontology, we fix prefix errors where `"PR"` is recorded as `"PRO"`. If you save the ontology without running the reasoner and reload it, the fix remains. If you open it after running ELK, the fix has been reversed. 


- <u>Example 2</u>: When we create the human subset of the Protein Ontology we verify that it contains only a single large connected component. If you re-calculate the number of connected components after running ELK, there will be three components.  

Luckily, the merged ontologies are not logically verified using a reasoner, thus the version used to build knowledge graphs remains free of these errors.

In [8]:
# instantiate and set-up class
ont_data = OntologyCleaner('', '', '', write_location)

# updating ontology info dictionary
ont_data.ontology_info = {k.split('/')[-1]: {} for k, v in ont_data.ontology_info.items()}

# set owl tools location
ont_data.owltools_location = '../pkt_kg/libs/owltools'

In [ ]:
# clean data
for ont in ont_data.ontology_info.keys():
    print('\n#### Processing Ontology: {} ####'.format(ont.upper()))
    ont_data.ont_file_location = ont
    ont_data.ont_graph = Graph().parse(ont_data.temp_dir + '/' + ont_data.ont_file_location)
    
    # get starting statistics
    ont_data.updates_ontology_reporter()
    
    # clean ontologies
    ont_data.fixes_ontology_parsing_errors()
    ont_data.fixes_identifier_errors()
    ont_data.removes_deprecated_obsolete_entities()
    ont_data.fixes_punning_errors()
    
    # run cleaned ontology through the elk reasoner
    logically_verifies_cleaned_ontologies(ont_data.ont_graph,
                                          ont_data.temp_dir,
                                          ont_data.ont_file_location,
                                          ont_data.owltools_location)

    # verifies no errors caused during cleaning
#     ontology_file_formatter(ont_data.temp_dir, '/' + ont_data.ont_file_location, ont_data.owltools_location)
    
    # read in cleaned, verified, and updated ontology containing inference
    print('Reading in Cleaned Ontology -- Needed to Calculate Final Statistics')
    ont_data.ont_graph = Graph().parse(ont_data.temp_dir + '/' + ont_data.ont_file_location)
    
    # get finishing statistics
    ont_data.updates_ontology_reporter()

<br>

***
### MERGED ONTOLOGIES <a class="anchor" id="merge-ontologies"></a>
***

**Purpose:** In this step, the [OWLTools](https://github.com/owlcollab/owltools) library is used to merge the directory of cleaned ontology files into a single ontology file. Then, the following cleaning steps are performed: (1) Identifier Errors; (2) Duplicate Classes; (3) Duplicate Class Concepts; and (4) Punning Errors.  

**Inputs:** A directory of ontology files (`.owl`)

**Outputs:** `PheKnowLator_MergedOntologies.owl`


In [ ]:
print('Merge Clean Ontology Data')
ont_data.ont_file_location = ont_data.merged_ontology_filename

# reorder list of ontology files to prepare for merging
onts = [ont_data.temp_dir + '/' + x for x in list(ont_data.ontology_info.keys())
        if x != ont_data.merged_ontology_filename]

# merge ontologies
merges_ontologies(onts, ont_data.temp_dir + '/', ont_data.ont_file_location, ont_data.owltools_location)

In [ ]:
# load merged ontologies into RDF Lib Graph object
print('Loading Merged Ontology Data')
ont_data.ont_graph = Graph().parse(ont_data.temp_dir + '/' + ont_data.ont_file_location)

# add merged ontology to dict
ont_data.ontology_info[ont_data.ont_file_location] = {}

# get stats on merged ontologies
print(derives_graph_statistics(ont_data.ont_graph))

### Clean Merged Ontologies
🤔 *IMPORTANT*🤔  Please note there are a few decisions that can made be made at this point that you may want to consider. For our monthly `PheKnowLator` builds, we prefer to use Entrez gene identifiers. If you have run the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook without makeing updates, you would have also committed yourself to using this type of gene identifier. If you have not done with this and do not want to use Entrez gene, but rather prefer to use what the ontologies provide, please comment out `ont_data.normalizes_existing_classes()` below.

In [ ]:
# get starting statistics
ont_data.updates_ontology_reporter()

# clean merged ontologies
ont_data.fixes_identifier_errors()
ont_data.normalizes_duplicate_classes()
ont_data.normalizes_existing_classes()
ont_data.fixes_punning_errors()

# get finishing statistics
print(derives_graph_statistics(ont_data.ont_graph))
ont_data.updates_ontology_reporter()

### Output and Save Results
The cleaned merged ontology file is saved to the `resources/knowledge_graphs` directory where it can be detected by the `PheKnowLator` algorithm during the build process.

In [ ]:
print('Save and Format Merged Ontology Data')
ont_data.ont_graph.serialize(destination=knowledge_graphs_location + '/' + ont_data.ont_file_location, format='xml')
ontology_file_formatter(knowledge_graphs_location, '/' + ont_data.ont_file_location, ont_data.owltools_location)

#### Save Ontology Cleaning Results  
To view the results of the ontology cleaning process print the `ont_data.ontology_info` dictionary. This dictionary is keyed by ontology filename and contains a separate dictionary for each ontology with descriptions of the results for each error check that is performed at the individual- and merged-ontology level. The results are also saved to `resources/ontologies/ontology_cleaning_report.txt`.

In [14]:
# save output locally
ont_order = sorted([x for x in ont_data.ontology_info.keys() if not x.startswith('Phe')]) + [ont_data.ont_file_location]
with open(ont_data.temp_dir + '/ontology_cleaning_report.txt', 'w') as o:
    o.write('=' * 50 + '\n{}'.format('ONTOLOGY CLEANING REPORT'))
    o.write('\n{}\n'.format(str(datetime.datetime.utcnow().strftime('%a %b %d %X UTC %Y'))) + '=' * 50 + '\n\n')
    for key in ont_order:
        o.write('\n\n\nONTOLOGY: {}\n'.format(key)); o.write('*' * (len(key) + 10) + '\n\n')
        x = ont_data.ontology_info[key]
        if 'Original GCS URL' in x.keys(): o.write('\t- Original GCS URL: {}\n'.format(x['Original GCS URL']))
        if 'Processed GCS URL' in x: o.write('\t- Processed GCS URL: {}\n'.format(x['Processed GCS URL']))
        o.write('\t- Statistics Before Cleaning:\n\t\t- {}\n'.format(x['Starting Statistics']))
        o.write('\t- Statistics After Cleaning:\n\t\t- {}\n'.format(x['Final Statistics']))
        if 'ValueErrors' in x.keys():
            if isinstance( x['ValueErrors'], str): o.write('\t- Value Errors (n=1):\n\t\t- {}\n'.format(x['ValueErrors']))
            else:
                for i in x['ValueErrors']: o.write('\t\t- {}\n'.format(str(i)))
        else: o.write('\t- Value Errors: 0\n')     
        if x['IdentifierErrors'] != 'None':
            o.write('\t- Identifier Errors (n={}):\n'.format(len(x['IdentifierErrors'].split(', '))))
            for i in x['IdentifierErrors'].split(', '): o.write('\t\t- {}\n'.format(str(i)))
        else: o.write('\t- Identifier Errors: 0\n')
        if 'PheKnowLator_MergedOntologies' not in key:
            if x['Deprecated'] != 'None':
                o.write('\t- Deprecated Classes (n={}):\n'.format(len(x['Deprecated'])))
                for i in x['Deprecated']: o.write('\t\t- {}\n'.format(str(i)))
            else: o.write('\t- Deprecated Classes: 0\n') 
            if x['Obsolete'] != 'None':
                o.write('\t- Obsolete Classes (n={}):\n'.format(len(x['Obsolete'])))
                for i in x['Obsolete']: o.write('\t\t- {}\n'.format(str(i)))
            else: o.write('\t- Obsolete Classes: 0\n')
        o.write('\t- Punning Errors:\n')
        if x['PunningErrors - Classes'] != 'None':
            o.write('\t\t- Classes (n={}):\n'.format(len(x['PunningErrors - Classes'].split(', '))))
            for i in x['PunningErrors - Classes'].split(', '): o.write('\t\t\t- {}\n'.format(i))
        else: o.write('\t\t- Classes: 0\n')
        if x['PunningErrors - ObjectProperty'] != 'None':
            o.write('\t\t- Object Properties (n={}):\n'.format(len(x['PunningErrors - ObjectProperty'].split(', '))))
            for i in x['PunningErrors - ObjectProperty'].split(', '): o.write('\t\t\t- {}\n'.format(i))
        else: o.write('\t\t- Object Properties: 0\n')
        if 'Normalized - Duplicates' in x.keys():
            o.write('\t- Normalization:\n')
            if x['Normalized - Duplicates'] != 'None':
                o.write('\t\t- Existing Entity Normalization (n={}):\n'.format(len(x['Normalized - Duplicates'].split(', '))))
                for i in x['Normalized - Duplicates'].split(', '): o.write('\t\t\t- {}\n'.format(i))
            else: o.write('\t\t- Entity Normalization: 0\n')
            if x['Normalized - Gene IDs'] != 'None': o.write('\t\t- Normalized HGNC IDs: {}\n'.format(x['Normalized - Gene IDs']))
            if x['Normalized - NonOnt'] != 'None': o.write('\t\t- Other Classes that May Need Normalization: {}\n'.format(x['Normalized - NonOnt']))
            if x['Normalized - Dep'] != 'None':
                o.write('\t\t- Deprecated Ontology HGNC Identifiers Needing Alignment (n={}):\n'.format(len(x['Normalized - Dep'])))
                for i in x['Normalized - Dep']: o.write('\t\t- {}\n'.format(i))
            else: o.write('\t\t- Deprecated Ontology HGNC Identifiers Needing Alignment: 0\n')
o.close()

<br>

***
### CREATE MERGED ONTOLOGIES GRAPH <a class="anchor" id="graph-ontologies"></a>
***

We run OWL-NETS via PheKnowLator on each ontology file.

In [ ]:
!echo "nan-nan, resources/edge_data/nan.txt" > $resource_data_location/edge_source_list.txt
!echo "nan-nan|;;|class-class|RO_0002001|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|0;1|None|None|None" > $resource_data_location/resource_info.txt
!echo "nan  nan" > $edge_data_location/nan.txt

import os
ontology_kg = []
os.rename(ontology_data_location + 'PheKnowLator_MergedOntologies.owl', ontology_data_location + 'merged_with_imports.owl')
for i in ['ro', 'chebi', 'pr', 'mondo', 'go', 'pw', 'so', 'hp', 'ext', 'vo', 'clo']:
    shutil.copy(ontology_data_location + i + '_with_imports.owl', resource_data_location + 'knowledge_graphs/PheKnowLator_MergedOntologies.owl')
    !cd ..; source ../pky/bin/activate; python3 main.py
    os.rename(resource_data_location + 'knowledge_graphs/PheKnowLator_v3.1.2_full_subclass_inverseRelations_OWLNETS_SUBCLASS_purified.nt',
              ontology_data_location + i + '.nt')
    df = pd.read_csv(ontology_data_location + i + '.nt', sep=' ', header=None)
    df['Source'] = i.upper()
    ontology_kg.append(df)
    for f in os.listdir(resource_data_location + 'knowledge_graphs/'):
        os.remove(resource_data_location + 'knowledge_graphs/' + f)

ontology_kg = pd.concat(ontology_kg)
ontology_kg['Source'] = ontology_kg['Source'].replace('EXT', 'Uberon')
ontology_kg['Source'] = ontology_kg['Source'].replace('CHEBI', 'ChEBI')
ontology_kg['Source'] = ontology_kg['Source'].replace('MONDO', 'Mondo')
ontology_kg['Source'] = ontology_kg['Source'].replace('HP', 'HPO')
ontology_kg['Source'] = ontology_kg['Source'].replace('PR', 'PRO')
ontology_kg.head(n=3)

We run OWL-NETS also on the merged ontology graph. This ensures to type "extra" edges introduced by OWL-NETS to ensure a single connected component. 

In [ ]:
shutil.copy(ontology_data_location + 'merged_with_imports.owl', resource_data_location + 'knowledge_graphs/PheKnowLator_MergedOntologies.owl')
!cd ..; source ../pky/bin/activate; python3 main.py
os.rename(resource_data_location + 'knowledge_graphs/PheKnowLator_v3.1.2_full_subclass_inverseRelations_OWLNETS_SUBCLASS_purified.nt',
        ontology_data_location + 'merged.nt')
merged_ontology = pd.read_csv(ontology_data_location + 'merged.nt', sep=' ', header=None)
for f in os.listdir(resource_data_location + 'knowledge_graphs/'):
    os.remove(resource_data_location + 'knowledge_graphs/' + f)
merged_ontology = merged_ontology[~merged_ontology[0].str.startswith('<https://o')] # This refers to orcid Ids
merged_ontology = merged_ontology[~merged_ontology[2].str.startswith('<https://o')]
merged_ontology = merged_ontology[~merged_ontology[0].str.startswith('<https://search.clinicalgenome.org/kb/conditions/MONDO:')] # Wrong Mondo IDs
merged_ontology = merged_ontology[~merged_ontology[2].str.startswith('<https://search.clinicalgenome.org/kb/conditions/MONDO:')]
merged_ontology[0] = merged_ontology[0].str.replace('http://www.genenames.org/cgi-bin/gene_symbol_report?hgnc_id=','http://identifiers.org/hgnc/')
merged_ontology[2] = merged_ontology[2].str.replace('http://www.genenames.org/cgi-bin/gene_symbol_report?hgnc_id=','http://identifiers.org/hgnc/')
merged_ontology_kg = pd.merge(merged_ontology, ontology_kg, how='left', on=[0, 1, 2])
merged_ontology_kg['Source'] = merged_ontology_kg['Source'].fillna('OWLNETS')
merged_ontology_kg['Source'] = merged_ontology_kg['Source'].replace('None', 'OWLNETS')

mask = ( # Fix mislabeled PRO
    merged_ontology_kg[0].str.startswith('<http://purl.obolibrary.org/obo/PR_') &
    (merged_ontology_kg[1] == '<http://purl.obolibrary.org/obo/pr#has_gene_template>') &
    merged_ontology_kg[2].str.startswith('<http://identifiers.org/hgnc/')
)
merged_ontology_kg.loc[mask, 'Source'] = 'PRO'

merged_ontology_kg = merged_ontology_kg.drop(columns=['3_x','3_y'])
merged_ontology_kg.head(n=3)

We still noticed the presence of HGNC identifiers and DOID after running OWL-NETS. We map them on NCBI Entrez identifiers and Mondo terms.

In [ ]:
merged_ontology_kg[0].str[:20].unique()

In [ ]:
mondo_graph = Graph().parse(ontology_data_location + 'mondo_with_imports.owl')

mondo_dbxref = gets_ontology_class_dbxrefs(mondo_graph)[0]

# Fix DOIDs (substitute : with _)
mondo_dict = {str(k).replace(':','_').upper(): {str(i).split('/')[-1].replace(':','_') for i in v}
              for k, v in mondo_dbxref.items() if 'doid' in str(k)}
list({**mondo_dict}.items())[:5]

In [10]:
with open(processed_data_location + 'DOID_MONDO_MAP.txt', 'w') as outfile:
    for k, v in mondo_dict.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

In [ ]:
doid_mondo_map = pd.read_csv(processed_data_location+'DOID_MONDO_MAP.txt', header=None, delimiter='\t')
doid_mondo_map[1] = doid_mondo_map[1].str.split(', ')
doid_mondo_map = doid_mondo_map.explode(1)
doid_mondo_map[0] = "<http://purl.obolibrary.org/obo/" + doid_mondo_map[0].astype(str) + ">"
doid_mondo_map[1] = "<http://purl.obolibrary.org/obo/" + doid_mondo_map[1].astype(str) + ">"
doid_mondo_map.head(n=3)

In [ ]:
merged_ontology_kg = merged_ontology_kg.rename(columns={0: 'Subject', 1: 'Predicate', 2: 'Object'})
merged_ontology_kg = pd.merge(merged_ontology_kg, doid_mondo_map, left_on='Subject', right_on=0, how='left')
merged_ontology_kg[1] = merged_ontology_kg[1].fillna(merged_ontology_kg['Subject'])
merged_ontology_kg.drop(columns=[0, 'Subject'], inplace=True)
merged_ontology_kg = merged_ontology_kg.rename(columns={1: 'Subject'})
merged_ontology_kg = pd.merge(merged_ontology_kg, doid_mondo_map, left_on='Object', right_on=0, how='left')
merged_ontology_kg[1] = merged_ontology_kg[1].fillna(merged_ontology_kg['Object'])
merged_ontology_kg.drop(columns=[0, 'Object'], inplace=True)
merged_ontology_kg = merged_ontology_kg.rename(columns={1: 'Object'})
merged_ontology_kg = merged_ontology_kg.rename(columns={'Subject': 0, 'Predicate': 1, 'Object':2})
merged_ontology_kg = merged_ontology_kg[~merged_ontology_kg[0].str.startswith('<http://purl.obolibrary.org/obo/DOID_')]
merged_ontology_kg = merged_ontology_kg[~merged_ontology_kg[2].str.startswith('<http://purl.obolibrary.org/obo/DOID_')]
merged_ontology_kg.head(n=3)

In [ ]:
hgnc_ncbi = pd.read_csv(processed_data_location + 'Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt',
                        sep='\t', dtype={'entrez_id':'Int64'})[['hgnc_id', 'entrez_id']]
hgnc_ncbi = hgnc_ncbi[(hgnc_ncbi['entrez_id'].astype(str) != '<NA>') & (~hgnc_ncbi['hgnc_id'].isna())]
hgnc_ncbi['hgnc_id'] = hgnc_ncbi['hgnc_id'].astype(str).str.replace('.0', '')
hgnc_ncbi['hgnc_id'] = hgnc_ncbi['hgnc_id'].str.replace('HGNC:', '')
hgnc_ncbi['hgnc_id'] = '<http://identifiers.org/hgnc/' + hgnc_ncbi['hgnc_id'].astype(str) + '>'
hgnc_ncbi['entrez_id'] = '<http://www.ncbi.nlm.nih.gov/gene/' + hgnc_ncbi['entrez_id'].astype(str) + '>'
hgnc_ncbi.drop_duplicates(inplace=True)
hgnc_ncbi.rename(columns={'hgnc_id': 0, 'entrez_id': 1}, inplace=True)
hgnc_ncbi.head(n=3)

In [ ]:
merged_ontology_kg = merged_ontology_kg.rename(columns={0: 'Subject', 1: 'Predicate', 2: 'Object'})
merged_ontology_kg = pd.merge(merged_ontology_kg, doid_mondo_map, left_on='Subject', right_on=0, how='left')
merged_ontology_kg[1] = merged_ontology_kg[1].fillna(merged_ontology_kg['Subject'])
merged_ontology_kg.drop(columns=[0, 'Subject'], inplace=True)
merged_ontology_kg = merged_ontology_kg.rename(columns={1: 'Subject'})
merged_ontology_kg = pd.merge(merged_ontology_kg, doid_mondo_map, left_on='Object', right_on=0, how='left')
merged_ontology_kg[1] = merged_ontology_kg[1].fillna(merged_ontology_kg['Object'])
merged_ontology_kg.drop(columns=[0, 'Object'], inplace=True)
merged_ontology_kg = merged_ontology_kg.rename(columns={1: 'Object'})
merged_ontology_kg = merged_ontology_kg.rename(columns={'Subject': 0, 'Predicate': 1, 'Object':2})
merged_ontology_kg = merged_ontology_kg[~merged_ontology_kg[0].str.startswith('<http://identifiers.org/hgnc/')]
merged_ontology_kg = merged_ontology_kg[~merged_ontology_kg[2].str.startswith('<http://identifiers.org/hgnc/')]
merged_ontology_kg.head(n=3)

We now map predicate identifiers to human-readable labels.

In [ ]:
ety_df = pd.DataFrame(set(merged_ontology_kg[1]),columns=["name"])
ety_df.head()

In [68]:
# split camel case strings, e.g., "overexpressedIn" --> "overexpressed in"
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# automatically match all OBO items for the specified ontologies
hdr = {'Accept': 'application/json'}
ontos = ["ro","vo","clo","mondo","ogg","cl","mf","bspo"]
tomatch = "http://purl.obolibrary.org/obo/"

def uri2etype(uri: str)->Union[str,None]:
    label = None
    for oy in ontos:
        baseuri = f"https://www.ebi.ac.uk/ols4/api/ontologies/{oy}/properties?iri={uri[1:-1]}"
        try:
            res = requests.get(baseuri,hdr).json()
            label=res['_embedded']['properties'][0]['label']
            label=label
        except:
            pass

    #if ("http://www.w3.org/1999/02/22-rdf-syntax-ns#type" in uri):   
     #   label = "Type"  
    #el
    if ("http://www.w3.org/2000/01/rdf-schema#subClassOf" in uri):   
        label = "SubClassOf" 
    # new manually set edges by splitting over the # symbol of the uri
    elif ("http://semanticscience.org/resource/SIO_000420" in uri):   
        label = "Has expression"
    elif ("http://purl.obolibrary.org/obo/CLO_0054408" in uri):   
        label = "Overexpresses gene"
    elif ("http://purl.obolibrary.org/obo/ddanat#part_of" in uri):   
        label = "Part of"
    elif ("http://purl.obolibrary.org/obo/ddanat#develops_from" in uri):   
        label = "Develops from"
    elif ("http://purl.obolibrary.org/obo/activates" in uri):   
        label = "Activates"
    elif ("http://purl.obolibrary.org/obo/GOREL_0012006" in uri):   
        label = "Results in maintenance of"
    elif ("http://purl.obolibrary.org/obo/GOREL_0000040" in uri):   
        label = "Results in"
    elif ("http://purl.obolibrary.org/obo/CLO_0054409" in uri):   
        label = "Adenoma formation induced by cell lineage cells in mice"
    elif ("http://purl.obolibrary.org/obo/uberon/core" in uri) or \
         ("http://purl.obolibrary.org/obo/mondo" in uri) or \
         ("http://purl.obolibrary.org/obo/so" in uri) or \
         ("http://purl.obolibrary.org/obo/envo" in uri) or \
         ("http://purl.obolibrary.org/obo/pr" in uri) or \
         ("http://purl.obolibrary.org/obo/pato" in uri) or \
         ("http://purl.obolibrary.org/obo/pw" in uri) or \
         ("http://purl.obolibrary.org/obo/exo.obo" in uri) or \
         ("http://purl.obolibrary.org/obo/cl" in uri) or \
         ("http://purl.obolibrary.org/obo/nbo" in uri) or \
         ("http://purl.obolibrary.org/obo/MF" in uri) or \
         ("http://www.obofoundry.org/ro" in uri) or \
         ("http://purl.obolibrary.org/obo/chebi" in uri):
            label = uri[1:-1].split("#")[1]
            label = '_'.join(camel_case_split(label))
    else:
        pass
    
    return label.replace(' ','_').lower()

In [ ]:
etypes_list = []
for u in tqdm(ety_df["name"].values):
    ety = uri2etype(u)
    etypes_list.append(ety)

ety_df.loc[:,"type"] = etypes_list
print("Unassigned edge types:",ety_df.type.isna().sum())
ety_df.to_csv('../ety.csv',index=False)
#ety_df = pd.read_csv('../ety.csv')
ety_df.head(n=3)

We fix edge type semantics according to RO and ensure every inverse relationship is properly added.

In [ ]:
ro_graph = Graph().parse(ontology_data_location + 'ro_with_imports.owl')

with open(relations_data_location + 'INVERSE_RELATIONS.txt', 'w') as outfile:
    outfile.write('Relation' + '\t' + 'Inverse_Relation' + '\n')
    for s, p, o in tqdm(ro_graph):
        if 'owl#inverseOf' in str(p):
            if 'RO' in str(s) and 'RO' in str(o):
                outfile.write(str(s.split('/')[-1]) + '\t' + str(o.split('/')[-1]) + '\n')
                outfile.write(str(o.split('/')[-1]) + '\t' + str(s.split('/')[-1]) + '\n')

# Fix RO_0002529's inverseOf issue (RO_0002529 is wrongly stored as inverseOf RO_0002529 instead of RO_0002528)
ro_data = pd.read_csv(relations_data_location + 'INVERSE_RELATIONS.txt', header=0, delimiter='\t').drop_duplicates()
ro_data.loc[ro_data['Relation'] == 'RO_0002529', 'Inverse_Relation'] = 'RO_0002528'
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_0002528'], 'Inverse_Relation': ['RO_0002529']})], ignore_index=True)

# We also specify symmetric relations (e.g., RO_0002434 inverseOf RO_0002434)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_0002434'], 'Inverse_Relation': 'RO_0002434'})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_HOM0000016'], 'Inverse_Relation': 'RO_HOM0000016'})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_0002436'], 'Inverse_Relation': ['RO_0002436']})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_HOM0000000'], 'Inverse_Relation': ['RO_HOM0000000']})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_0002526'], 'Inverse_Relation': ['RO_0002526']})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['RO_0002325'], 'Inverse_Relation': ['RO_0002325']})], ignore_index=True)
# We also specify 'part of' as inverse of 'has part'
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['BFO_0000050'], 'Inverse_Relation': ['BFO_0000051']})], ignore_index=True)
ro_data = pd.concat([ro_data, pd.DataFrame({'Relation': ['BFO_0000051'], 'Inverse_Relation': ['BFO_0000050']})], ignore_index=True)
ro_data.to_csv(relations_data_location + 'INVERSE_RELATIONS.txt', sep='\t', index=False)

print(ro_data.head(n=3))

results = {str(x[2]).lower(): str(x[0]) for x in ro_graph if '/RO_' in str(x[0]) and 'label' in str(x[1]).lower()}

with open(relations_data_location + 'RELATIONS_LABELS.txt', 'w') as outfile:
    outfile.write('Label' + '\t' + 'Relation' + '\n')
    for k, v in results.items():
        outfile.write(str(v).split('/')[-1] + '\t' + str(k) + '\n')

ro_data_label = pd.read_csv(relations_data_location + 'RELATIONS_LABELS.txt', header=0, delimiter='\t')
# We also add the 'part of' relation (BFO_0000050) and its inverse 'has part' (BFO_0000051)
ro_data_label = pd.concat([ro_data_label, pd.DataFrame({'Label': ['BFO_0000050'], 'Relation': ['part of']})], ignore_index=True)
ro_data_label = pd.concat([ro_data_label, pd.DataFrame({'Label': ['BFO_0000051'], 'Relation': ['has part']})], ignore_index=True)
ro_data_label['Relation'] = ro_data_label['Relation'].str.replace("is substitutent group from","is substituent group from")
ro_data_label['Relation'] = ro_data_label['Relation'].str.lower().str.replace(" ", "_")
ro_data_label.to_csv(relations_data_location + 'RELATIONS_LABELS.txt', sep='\t', index=False)

print(ro_data_label.head(n=3))
ro_data_inverse = ro_data.merge(ro_data_label, left_on='Relation', right_on='Label')
ro_data_inverse = ro_data_inverse.merge(ro_data_label, left_on='Inverse_Relation', right_on='Label')[
    ['Relation', 'Relation_y']].rename(columns={'Relation_y': 'Inverse_Relation'})
ro_data_inverse.head(n=6)

In [77]:
ety_df['type'] = ety_df['type'].replace('disease_arises_from_feature','disease_has_basis_in_feature')
ety_df['type'] = ety_df['type'].replace('non_functional_homolog_of','in_non_functional_homology_relationship_with')
ety_df['type'] = ety_df['type'].replace('has_gene_template','gene_product_of')
ety_df['type'] = ety_df['type'].replace('variant_of','evolutionary_variant_of')
ety_df['type'] = ety_df['type'].replace('in_response_to','realized_in_response_to')

In [ ]:
merged_ontology_kg = pd.merge(merged_ontology_kg, ety_df, left_on=1, right_on='name').drop(columns=['name',1], axis=1)
merged_ontology_kg.head(n=3)

In [ ]:
merged_ontology_kg_inverse = merged_ontology_kg.rename(columns={2: 0, 0: 2})
merged_ontology_kg_inverse = merged_ontology_kg_inverse.merge(
    ro_data_inverse, left_on='type',right_on='Relation', how='left').drop(columns=['type']).rename(columns={'Inverse_Relation': 'type'})
merged_ontology_kg = pd.concat([merged_ontology_kg, merged_ontology_kg_inverse]).drop_duplicates()
merged_ontology_kg = merged_ontology_kg.groupby([0, 'type', 2]).agg({'Source': set}).reset_index()
merged_ontology_kg.head(n=3)

In [ ]:
merged_ontology_kg[0] = merged_ontology_kg[0].apply(lambda x: x[1:-1] if x.startswith('<') and x.endswith('>') else x)
merged_ontology_kg[2] = merged_ontology_kg[2].apply(lambda x: x[1:-1] if x.startswith('<') and x.endswith('>') else x)

# Remove OWLNETS if it is not the only source
merged_ontology_kg['Source'] = merged_ontology_kg['Source'].apply(
    lambda source_set: source_set - {'OWLNETS'} if (len(source_set) > 1 and 'OWLNETS' in source_set) else source_set
)

merged_ontology_kg['Source'] = merged_ontology_kg['Source'].apply(lambda x: list(eval(x)))
merged_ontology_kg.to_csv(ontology_data_location + 'merged_ontology_kg.txt', sep='\t', header=False, index=False)


<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```
```
@misc{cavalleri_e_2024_rna_kg,
  author       = {Cavalleri, E},
  title        = {RNA-KG},
  year         = 2024,
  doi          = {10.5281/zenodo.10078876},
  url          = {https://doi.org/10.5281/zenodo.10078876}
}
```